Features_df importieren

In [105]:
import pandas as pd

features_df = pd.read_pickle("../data/processed/features.pkl")

X definieren

In [106]:
X = features_df[['month_sin',
                 'month_cos',
                 'day_of_week_sin',
                 'day_of_week_cos',
                 'minute_of_day_sin',
                 'minute_of_day_cos',
                 'wind_dir_sin',
                'wind_dir_cos',
                'wind_speed_kt',
                'is_wind_variable',
                'vis_ge_10km',
                'vis_category',
                'temperature_c',
                'spread_c',
                'qnh_hpa',
                'is_holiday',
                'scheduled_departures',
                'scheduled_arrivals',
                'avg_dep_delay_prev_slot_minutes',
                'avg_dep_delay_past_6h_minutes',
                'scheduled_B',
                'scheduled_C',
                'scheduled_D',
                'scheduled_E',
                'scheduled_F',
                'scheduled_rwy_concept',
                'arr_dep_ratio',
        ]]

Y definieren

In [107]:
y = features_df[['avg_dep_delay_slot_minutes']]

XGBoost und train_test_split importieren

In [108]:
import xgboost
from sklearn.model_selection import train_test_split

Daten in Trainings-, Validations- und Testset aufteilen

In [109]:
X_train, X_temp, y_train, y_temp = train_test_split(X,y,test_size=0.4,random_state=42,shuffle=False)
X_test, X_val, y_test, y_val = train_test_split(X_temp,y_temp,test_size=0.5,random_state=42,shuffle=False) 

'''
Daten werden aufgesplittet:
- 60% Training
- 20% Validierung
- 20% Test
'''

'\nDaten werden aufgesplittet:\n- 60% Training\n- 20% Validierung\n- 20% Test\n'

XGBRegressor importieren

In [110]:
from xgboost import XGBRegressor

Hyperparameter definieren

In [111]:
model = XGBRegressor(
    objective = 'reg:squarederror',
    enable_categorical = True,
    random_state = 42,
    early_stopping_rounds = 50,
    n_estimators = 100000,
    learning_rate = 0.05,
)

Modell trainieren

In [112]:
model.fit(X_train,y_train,eval_set=[(X_val,y_val)],verbose=0)

''' 
Modell aufgrund der Trainingsdaten trainieren.
Nach jeder Iteration wird überprüft, wie sich das Modell auf dem Validationsset verhält.
Wenn sich der Fehler auf dem Validationset für 50 Iterationen in Folge nicht verbessert, wird das Training abgebrochen (early stopping).
'''

' \nModell aufgrund der Trainingsdaten trainieren.\nNach jeder Iteration wird überprüft, wie sich das Modell auf dem Validationsset verhält.\nWenn sich der Fehler auf dem Validationset für 50 Iterationen in Folge nicht verbessert, wird das Training abgebrochen (early stopping).\n'

Statistische Tests importieren

In [113]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

Trainiertes Modell am Validation-Set anwenden

In [114]:
y_pred_val = model.predict(X_val)

Mean Absolute Error (MAE)

In [115]:
mae_y_val = mean_absolute_error(y_val,y_pred_val)
print("Mean Absolute Error (MAE) on Validation Set:", mae_y_val, "minutes")

Mean Absolute Error (MAE) on Validation Set: 5.073862075805664 minutes


Root Mean Square Error (RMSE)

In [116]:
rmse_y_val = root_mean_squared_error(y_val,y_pred_val)
print("Root Mean Square Error (RMSE) on Validation Set:", rmse_y_val, "minutes")

Root Mean Square Error (RMSE) on Validation Set: 10.439760208129883 minutes


R^2 Score

In [117]:
r2_y_val = r2_score(y_val,y_pred_val)
print("R^2 Score on Validation Set:", r2_y_val)


R^2 Score on Validation Set: 0.5075543522834778


Baseline Scores printen und als CSV abspeichern

In [120]:
baseline_scores_v1 = {
    "MAE": mae_y_val,
    "RMSE": rmse_y_val,
    "R^2": r2_y_val
}
print(baseline_scores_v1)
baseline_df = pd.DataFrame([baseline_scores_v1])
baseline_df.to_csv("../data/processed/baseline_model_scores_v1.0.csv")

{'MAE': 5.073862075805664, 'RMSE': 10.439760208129883, 'R^2': 0.5075543522834778}
